# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
df = spark.read.option("header", "true") \
               .option("delimiter", ";") \
               .option("multiline", "true") \
               .option("quote", "\"") \
               .option("escape", "\"") \
               .csv("/FileStore/Examen/Caso4/ds_jobs.csv")
df.printSchema()

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)



2. Eliminar duplicados

In [0]:
# El siguiente comando elimina duplicados. Como no hay duplicados (ya que el index es distinto en todos) no se va a eliminar nada. 
df = df.dropDuplicates()


3. Decidir que hacer con los datos faltantes 

Si faltan muchos datos de la empresa como en HireAI, yo eliminaría esas filas.

4. Decidir que hacer con los valores nulos

no hay nulos.

5. ¿Cuántos registros tiene el csv?

In [0]:
registros = df.count()
print(f"Numero de registros: {registros}")

Numero de registros: 678


6. Mostrar los valores únicos de `Job title` 

In [0]:
distinct_job = df.select("Job Title").distinct()
distinct_job.show(truncate=False)  

+----------------------------------------------+
|Job Title                                     |
+----------------------------------------------+
|Business Intelligence Analyst                 |
|Data Modeler                                  |
|Senior Research Statistician- Data Scientist  |
|Sr Data Scientist                             |
|Data Scientist/Machine Learning               |
|Data Scientist / Machine Learning Expert      |
|Associate Data Scientist                      |
|Medical Lab Scientist                         |
|Human Factors Scientist                       |
|Experienced Data Scientist                    |
|Data Analyst II                               |
|Data Scientist                                |
|Data Analyst                                  |
|Senior Analyst/Data Scientist                 |
|Data Scientist-Human Resources                |
|Data Scientist - Contract                     |
|Data Scientist - Risk                         |
|Business Intelligen

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
from pyspark.sql.functions import regexp_replace
df = df.withColumn("Salary Estimate",  regexp_replace(df["Salary Estimate"], "K", "000"))

8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
distinct_salary = df.select("Salary Estimate").distinct()
distinct_salary.show(truncate=False)  

+--------------------------------+
|Salary Estimate                 |
+--------------------------------+
|$79000-$106000 (Glassdoor est.) |
|$141000-$225000 (Glassdoor est.)|
|$112000-$116000 (Glassdoor est.)|
|$31000-$56000 (Glassdoor est.)  |
|$56000-$97000 (Glassdoor est.)  |
|$145000-$225000(Employer est.)  |
|$122000-$146000 (Glassdoor est.)|
|$101000-$165000 (Glassdoor est.)|
|$79000-$133000 (Glassdoor est.) |
|$99000-$132000 (Glassdoor est.) |
|$79000-$131000 (Glassdoor est.) |
|$90000-$109000 (Glassdoor est.) |
|$69000-$116000 (Glassdoor est.) |
|$79000-$147000 (Glassdoor est.) |
|$90000-$124000 (Glassdoor est.) |
|$91000-$150000 (Glassdoor est.) |
|$137000-$171000 (Glassdoor est.)|
|$71000-$123000 (Glassdoor est.) |
|$110000-$163000 (Glassdoor est.)|
|$75000-$131000 (Glassdoor est.) |
+--------------------------------+
only showing top 20 rows



9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
from pyspark.sql.functions import split, col
df = df.withColumn("Salary Estimate", regexp_replace(df["Salary Estimate"], r"\(.*?\)", ""))

10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
df = df.withColumn("Salary Estimate", regexp_replace(df["Salary Estimate"], "\\$", ""))
df = df.orderBy(split(col("Salary Estimate"),"-")[1].cast("int").desc())

11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
df = df.withColumn("Job Description", regexp_replace(col("Job Description"),"\n",""))

12. De la columna `Rating` muestre los valores unicos.

In [0]:
distinct_rating = df.select("Rating").distinct()
distinct_rating.show(truncate=False)  

+------+
|Rating|
+------+
|29    |
|42    |
|34    |
|28    |
|22    |
|35    |
|47    |
|43    |
|31    |
|27    |
|41    |
|38    |
|44    |
|33    |
|48    |
|32    |
|36    |
|37    |
|39    |
|50    |
+------+
only showing top 20 rows



13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
df = df.withColumn("Rating", regexp_replace(col("Rating"),"-1","0"))

14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
distinct_company = df.select("Company Name").distinct()
distinct_company.show(truncate=False)  

+-----------------------------------------+
|Company Name                             |
+-----------------------------------------+
|Healthfirst\n3.1                         |
|iRobot\n3.5                              |
|Triplebyte\n3.2                          |
|HG Insights\n4.2                         |
|Tower Health\n3.5                        |
|Guzman & Griffin Technologies (GGTI)\n4.4|
|Buckman\n3.5                             |
|XSELL Technologies\n3.6                  |
|ManTech\n4.2                             |
|Intuit - Data\n4.4                       |
|Novetta\n4.5                             |
|PNNL\n3.7                                |
|Analysis Group\n3.8                      |
|Affinity Solutions\n2.9                  |
|Old World Industries\n3.1                |
|1904labs\n4.7                            |
|Upside Business Travel\n4.1              |
|Novartis\n3.9                            |
|INFICON\n3.5                             |
|Insight Enterprises, Inc.\n4.2 

15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
df = df.withColumn("Company Name", split(col("Company Name"), "\n")[0])

16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
df = df.withColumn("City", split(col("Location"), ",")[0])
df = df.withColumn("State", regexp_replace(split(col("Location"), ",")[1], " ", ""))
df = df.drop("Location")



17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
headquarters = split(col("Headquarters"), ",")

df = df.withColumn("Headquarter State", headquarters[1]) 
df = df.withColumn("Headquarters", headquarters[0])


18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
distinct_headquarter = df.select("Headquarter State").distinct()
distinct_headquarter.show(truncate=False)  

+-----------------+
|Headquarter State|
+-----------------+
| WA              |
| MO              |
| Japan           |
| NE              |
| AZ              |
| TN              |
| MA              |
| NY              |
| OH              |
| IL              |
| FL              |
| LA              |
| CA              |
| Switzerland     |
| Singapore       |
| PA              |
| NJ              |
| GA              |
| VA              |
| WI              |
+-----------------+
only showing top 20 rows



19. Mostrar valores unicos del campo `Size`.

In [0]:
distinct_size = df.select("Size").distinct()
distinct_size.show(truncate=False)  

+-----------------------+
|Size                   |
+-----------------------+
|-1                     |
|5001 to 10000 employees|
|0                      |
|Unknown                |
|51 to 200 employees    |
|1001 to 5000 employees |
|501 to 1000 employees  |
|201 to 500 employees   |
|10000+ employees       |
|1 to 50 employees      |
+-----------------------+



20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
df = df.withColumn("Size", regexp_replace(col("Size"), "employee", ""))
df = df.withColumn("Size", regexp_replace(col("Size"), "s", ""))


21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
df = df.withColumn("Size", regexp_replace(col("Size"), " to ", "-"))
df = df.withColumn("Size", regexp_replace(col("Size"), r"-1", "Unknown"))
df = df.withColumn("Size", regexp_replace(col("Size"), " ", ""))

22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
df.printSchema()

ownership_distinct = df.select("Type of ownership").distinct()
ownership_distinct.show(truncate=False)

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter State: string (nullable = true)

+------------------------------+
|Type of ownership             |
+------------------------------+
|-1                            |
|Government                    |
|Subsidiary or Business Segment|
|Self-employed                 |
|Contract                      |
|Unknown                  

23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:

df = df.withColumn("Type of ownership", regexp_replace("Type of ownership", r"-1", "Unknown"))

24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:

df = df.withColumn("Type of ownership", regexp_replace("Type of ownership", r"Company - Public", "Public Company"))
df = df.withColumn("Type of ownership", regexp_replace("Type of ownership", r"Company - Private", "Private Company"))
df = df.withColumn("Type of ownership", regexp_replace("Type of ownership", r"Private Practice / Firm", "Private Company"))
df = df.withColumn("Type of ownership", regexp_replace("Type of ownership", r"Subsidiary or Business Segment", "Business"))
df = df.withColumn("Type of ownership", regexp_replace("Type of ownership", r"College / University", "Education"))

25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
df.printSchema()
df.select("Industry").distinct().show(truncate=False)

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter State: string (nullable = true)

+----------------------------------------+
|Industry                                |
+----------------------------------------+
|-1                                      |
|Investment Banking & Asset Management   |
|Insurance Carriers                      |
|Energy                         

26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
df = df.withColumn("Industry", regexp_replace("Industry", r"-1", "Not Available"))
df = df.withColumn("Industry", regexp_replace("Industry", r"&", "and")) 

df.select("Industry").distinct().orderBy("Industry").show(truncate=False)

+------------------------------------------+
|Industry                                  |
+------------------------------------------+
|Accounting                                |
|Advertising and Marketing                 |
|Aerospace and Defense                     |
|Architectural and Engineering Services    |
|Banks and Credit Unions                   |
|Biotech and Pharmaceuticals               |
|Cable, Internet and Telephone Providers   |
|Chemical Manufacturing                    |
|Colleges and Universities                 |
|Computer Hardware and Software            |
|Construction                              |
|Consulting                                |
|Consumer Electronics and Appliances Stores|
|Consumer Products Manufacturing           |
|Department, Clothing, and Shoe Stores     |
|Electrical and Electronic Manufacturing   |
|Energy                                    |
|Enterprise Software and Network Solutions |
|Express Delivery Services                 |
|Farm Supp

27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
df.printSchema()
df.select("Sector").distinct().show(truncate=False)

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter State: string (nullable = true)

+----------------------------+
|Sector                      |
+----------------------------+
|Health Care                 |
|-1                          |
|Education                   |
|Insurance                   |
|Information Technology      |
|Government                  |
|Oil, Gas, 

28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
df = df.withColumn("Sector", regexp_replace("Sector", r"-1", "Not Available"))
df = df.withColumn("Sector", regexp_replace("Sector", r"&", "and")) 

df.select("Sector").distinct().orderBy("Sector").show(truncate=False)

+------------------------------------+
|Sector                              |
+------------------------------------+
|Accounting and Legal                |
|Aerospace and Defense               |
|Agriculture and Forestry            |
|Biotech and Pharmaceuticals         |
|Business Services                   |
|Construction, Repair and Maintenance|
|Consumer Services                   |
|Education                           |
|Finance                             |
|Government                          |
|Health Care                         |
|Information Technology              |
|Insurance                           |
|Manufacturing                       |
|Media                               |
|Non-Profit                          |
|Not Available                       |
|Oil, Gas, Energy and Utilities      |
|Real Estate                         |
|Retail                              |
+------------------------------------+
only showing top 20 rows



29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
df.printSchema()
df.select("Revenue").distinct().orderBy("Revenue").show(truncate=False)

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Competitors: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter State: string (nullable = true)

+--------------------------------+
|Revenue                         |
+--------------------------------+
|$1 to $2 billion (USD)          |
|$1 to $5 million (USD)          |
|$10 to $25 million (USD)        |
|$10+ billion (USD)              |
|$100 to $500 million (USD)      |
|$2 to $5 

30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
df = df.withColumn("Revenue", regexp_replace("Revenue", r"-1", "N/A"))
df = df.withColumn("Revenue", regexp_replace("Revenue", r"Unknown / Non-Applicable", "N/A"))
df = df.withColumn("Revenue", regexp_replace("Revenue", r"Less than $1 million (USD)", "Less than 1"))
df = df.withColumn("Revenue", regexp_replace("Revenue", r"\$", "")) 
df = df.withColumn("Revenue", regexp_replace("Revenue", r"\(USD\)", "")) 

df.select("Revenue").distinct().orderBy("Revenue").show(truncate=False)

+-------------------------+
|Revenue                  |
+-------------------------+
|0                        |
|1 to 2 billion           |
|1 to 5 million           |
|10 to 25 million         |
|10+ billion              |
|100 to 500 million       |
|2 to 5 billion           |
|25 to 50 million         |
|5 to 10 billion          |
|5 to 10 million          |
|50 to 100 million        |
|500 million to 1 billion |
|Less than 1 million      |
|N/A                      |
+-------------------------+



31. Borrar el campo `Competitors`.

In [0]:
df = df.drop("Competitors")

32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
df = df.withColumn("min_salary", split("Salary Estimate", "-")[0].cast("int"))
df = df.withColumn("max_salary", split("Salary Estimate", "-")[1].cast("int"))
df = df.withColumn("avg_salary", (col("min_salary") + col("max_salary")) / 2)


33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
df.printSchema()
df.select("Founded").distinct().show(truncate=False)

root
 |-- index: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salary Estimate: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Company Name: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Founded: string (nullable = true)
 |-- Type of ownership: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Headquarter State: string (nullable = true)
 |-- min_salary: integer (nullable = true)
 |-- max_salary: integer (nullable = true)
 |-- avg_salary: double (nullable = true)

+-------+
|Founded|
+-------+
|2016   |
|2012   |
|1988   |
|2017   |
|2014   |
|2000   |
|1965   |
|1981   |
|1968   |
|2011   |
|1997   |
|1973   |
|1996   |
|1983   |
|1986   |
|2015   |
|1998   |
|1993

34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
df = df.withColumn("Founded", regexp_replace("Founded", r"-1", "2024"))

35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

In [0]:
from pyspark.sql.functions import current_date, year

current_year = year(current_date())
df = df.withColumn("company_age", current_year - df["Founded"])


36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

In [0]:
from pyspark.sql.functions import when
df = df.withColumn("Job Type", when(col("Job Title").rlike("(?i)sr|senior|lead|principal"), "Senior").when(col("Job Title").rlike("(?i)jr|jr\."), "Junior").otherwise("NA"))


37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
df.select("Job Type").distinct().show(truncate=False)

+--------+
|Job Type|
+--------+
|Senior  |
|NA      |
|Junior  |
+--------+



38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


In [0]:
from pyspark.sql.functions import col, when

skills = ["Python", "Excel"]

for skill in skills:
    skill_col = skill.replace(" ", "_")  
    df = df.withColumn(
        skill_col,
        when(
            col("Job Description").rlike(f"(?i)\\b{skill}\\b"),
            1
        ).otherwise(0)
    )



39. Exportar dataset final a csv

In [0]:
df.write.option("header", "true").csv("/FileStore/Examen/Caso4/dataset.csv")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2175953469282411>:1
----> 1 df.write.option("header", "true").csv("/FileStore/Examen/Caso4/dataset.csv")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1798, in DataFrameWriter.csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
   1779 self.mode(mode)
   1780 self._

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 

In [0]:
from pyspark.sql import functions as F

# Averiguar empresas con alta calificacion que pidan python
df.filter((col("Python") == 1) & (col("Rating") > 4)).select("Company Name", "Rating").show()


# Agrupar por el sector, contar las ocurrencias y calcular la media del salario, redondeada a 2 decimales
sector_salary_stats = df.groupBy("Sector").agg(
    F.count("Sector").alias("sector_count"),  
    F.round(F.avg("avg_salary"), 2).alias("average_salary") 
)
sector_salary_stats_sorted = sector_salary_stats.orderBy(["sector_count", "average_salary"], ascending=[False, False])
sector_salary_stats_sorted.show()




+--------------------+------+
|        Company Name|Rating|
+--------------------+------+
|       Aveshka, Inc.|    38|
|              Aptive|    35|
|Alaka`ina Foundat...|    36|
|               Roche|    41|
|  Maxar Technologies|    35|
|  Comtech Global Inc|    40|
|Southwest Researc...|    39|
|Smith Hanley Asso...|    45|
|  Maxar Technologies|    35|
|     Creative Circle|    36|
|Southwest Researc...|    39|
|Enterprise Soluti...|    38|
|              Mackin|    34|
|   State of Virginia|    32|
|         Edmunds.com|    34|
|              Criteo|    39|
|         AstraZeneca|    40|
| Oshkosh Corporation|    42|
|              Leidos|    35|
|        Cambridge FX|    35|
+--------------------+------+
only showing top 20 rows

+--------------------+------------+--------------+
|              Sector|sector_count|average_salary|
+--------------------+------------+--------------+
|Information Techn...|         188|     118909.57|
|   Business Services|         122|     129815.57|